# ESO SSA Query Example Notebook
This notebook demonstrates how to programmatically query the ESO Simple Spectral Access (SSA) service for spectra within a specified sky region, filter results by signal-to-noise ratio (SNR), and download the matching data products.

**Author:** Your Name  
**Date:** 2025-08-07


## Introduction
ESO provides a Simple Spectral Access (SSA) endpoint that allows users to search for spectroscopic data by sky position. In this example, we will:
1. Resolve a target name to coordinates using Astropy's `SkyCoord` and the CDS SESAME service.
2. Query the ESO SSA service within a cone around the target.
3. Filter the returned spectra by a minimum SNR threshold.
4. Download the high-SNR spectra to the local machine.


In [9]:
import astroquery # import astroquery
from astroquery.eso import Eso # import the ESO module from astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

eso = Eso() # create an instance of the ESO class 

astroquery version: 0.4.11.dev10245


In [ ]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

target = "NGC4993" # set the target 
coords = SkyCoord.from_name(target) # create a SkyCoord object from the name of the source 
radius = 0.5 *u.deg # set the radius of the search 

table_reduced = eso.query_surveys(cone_ra=coords.ra.value, 
                                cone_dec=coords.dec.value, 
                                cone_radius=radius.to("deg").value) # query the ESO archive for HAWKI data

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id", "release_description"][:3] # print the first 3 rows of the table
table_reduced

Resolving target NGC 4993...
Resolved coordinates (ICRS): 197.449 -23.384 (deg)


## Setup and Imports
First, we import the necessary Python packages:
- `pyvo` to access VO services
- `astropy.coordinates` and `astropy.units` for coordinate and unit handling
- `sys` and `urllib` for input/output and downloading files


In [2]:
import sys
import urllib.request
import pyvo as vo
from astropy.coordinates import SkyCoord
from astropy.units import Quantity

## Define the SSA Service
Specify the SSA endpoint URL and create the service object:

In [3]:
ssap_endpoint = "http://archive.eso.org/ssap"
ssap_service = vo.dal.SSAService(ssap_endpoint)
print(f"SSA endpoint: {ssap_endpoint}")

SSA endpoint: http://archive.eso.org/ssap


## Resolve Target Name
Use the CDS SESAME service to convert a target name to sky coordinates:

In [4]:
target = "NGC 4993"
diameter = 0.5  # degrees
print(f"Resolving target {target}...")
pos = SkyCoord.from_name(target)
print(f"Resolved coordinates (ICRS): {pos.to_string()} (deg)")

Resolving target NGC 4993...
Resolved coordinates (ICRS): 197.449 -23.384 (deg)


## Perform SSA Query
Search for spectra within the specified cone:

In [7]:
size = Quantity(diameter, unit="deg")
print(f"Querying SSA for a {diameter}° diameter around {target}...")
ssap_resultset = ssap_service.search(pos=pos.fk5, diameter=size)
ssap_resultset.to_table()

Querying SSA for a 0.5° diameter around NGC 4993...


access_estsize,access_url,APERTURE,COLLECTION,creation_type,CREATORDID,curation_reference,datalink_url,dataset_length,dataset_title,dp_id,em_bw,em_max,em_min,em_val,equinox,flux_ucd,FLUXCALIB,FORMAT,gal_lat,gal_lon,instrument_name,MTIME,obs_creator_name,positionJ2000,PUBDID,publication_date,publisher,rights,s_dec,s_ra,s_region,SNR,space_frame,spatial_resolution,SPECRP,t_elapsed,t_exptime,t_max,t_mid,t_min,TARGETNAME,WAVECALIB
kbyte,,arcsec,,,,,,,,,m,m,m,m,,,,,deg,deg,,,,deg,,,,,deg,deg,,,,deg,,s,s,d,d,d,,
int64,object,float64,object,object,object,object,object,int64,object,object,float64,float64,float64,float64,float64,object,object,object,float64,float64,object,object,object,float64[2],object,object,object,object,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,object,object
3075,https://dataportal.eso.org/dataPortal/file/ADP.2016-09-20T08:08:12.820,1.99998,FEROS,archival,ivo://eso.org/origfile?FEROS.2004-03-07T04:07:01.411_DRS_FERN_1.0_ESOSDP.fits,,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2016-09-20T08:08:12.820,189654,HD114149_r.FEROS.2004-03-07T04:07:01.411.1081.fits,ADP.2016-09-20T08:08:12.820,5.68959e-07,9.21735e-07,3.52776e-07,6.372555e-07,2000.0,,uncalibrated,application/x-fits-bintable,39.573305,308.153068,FEROS,2020-10-13T13:59:35.647Z,"CHARBONNEL, C.",197.234 .. -23.1209,ivo://eso.org/ID?ADP.2016-09-20T08:08:12.820,2016-11-07T08:26:00Z,ESO,public,-23.12092,197.233637,POSITION J2000 197.233637 -23.12092,86.0,FK5,--,48000.0,0.0009490699958405457,81.999,53071.17249318,53071.172018645,53071.17154411,HD114149,absolute
3075,https://dataportal.eso.org/dataPortal/file/ADP.2016-09-20T08:08:13.622,1.99998,FEROS,archival,ivo://eso.org/origfile?FEROS.2004-03-06T07:10:25.662_DRS_FERN_1.0_ESOSDP.fits,,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2016-09-20T08:08:13.622,189656,HD114098_r.FEROS.2004-03-06T07:10:25.662.1081.fits,ADP.2016-09-20T08:08:13.622,5.68965e-07,9.21741e-07,3.52776e-07,6.372585e-07,2000.0,,uncalibrated,application/x-fits-bintable,39.593553,308.045421,FEROS,2020-10-13T13:59:35.647Z,"CHARBONNEL, C.",197.142 .. -23.108,ivo://eso.org/ID?ADP.2016-09-20T08:08:13.622,2016-11-07T08:26:00Z,ESO,public,-23.10798,197.141857,POSITION J2000 197.14185700000002 -23.10798,79.4,FK5,--,48000.0,0.004652780000469647,401.999,53070.30356091,53070.30123452,53070.29890813,HD114098,absolute
1330,https://dataportal.eso.org/dataPortal/file/ADP.2017-08-11T02:12:48.569,0.9,XSHOOTER,archival,ivo://eso.org/origfile?XS_SFLX_200270940_2013-06-03T23:39:09.754_S0.9x11_1x2_VIS_NOD.fits,,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2017-08-11T02:12:48.569,24318,Q1311-tellurHIP64271A0V_200270940_2013-06-03T23:39:09.754_S0.9x11_1x,ADP.2017-08-11T02:12:48.569,4.8634e-07,1.02e-06,5.336599999999999e-07,7.768299999999999e-07,2000.0,,absolute,application/x-fits-bintable,39.041556,308.512203,XSHOOTER,2020-10-13T14:00:43.807Z,"VESTERGAARD, MARIANNE",197.588 .. -23.6251,ivo://eso.org/ID?ADP.2017-08-11T02:12:48.569,2017-08-11T02:17:23Z,ESO,public,-23.62509,197.587964,POSITION J2000 197.587964 -23.62509,303.0,FK5,--,8935.0,0.001320470000791829,16.0,56446.98685003,56446.986189795,56446.98552956,Q1311-tellur HIP64271 A0V,absolute
1359,https://dataportal.eso.org/dataPortal/file/ADP.2017-08-11T02:12:48.733,0.9,XSHOOTER,archival,ivo://eso.org/origfile?XS_SFLX_200270940_2013-06-03T23:39:13.415_S0.9x11_NIR_NOD.fits,,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2017-08-11T02:12:48.733,24750,Q1311-tellurHIP64271A0V_200270940_2013-06-03T23:39:13.415_S0.9x11_NI,ADP.2017-08-11T02:12:48.733,1.48494e-06,2.47896e-06,9.9402e-07,1.73649e-06,2000.0,,absolute,application/x-fits-bintable,39.041556,308.512203,XSHOOTER,2020-10-13T14:00:43.807Z,"VESTERGAARD, MARIANNE",197.588 .. -23.6251,ivo://eso.org/ID?ADP.2017-08-11T02:12:48.733,2017-08-11T02:17:23Z,ESO,public,-23.62509,197.587964,POSITION J2000 197.587964 -23.62509,434.4,FK5,--,5573.0,0.0015700799995101988,60.0,5

## Inspect and Filter Results
Define the fields to display and filter by minimum SNR:

In [ ]:
fields = ["COLLECTION", "TARGETNAME", "s_ra", "s_dec", "APERTURE",
          "em_min", "em_max", "SPECRP", "SNR", "t_min", "t_max",
          "CREATORDID", "access_url"]
min_SNR = 70

print(f"Filtering spectra with SNR > {min_SNR}...")
count_high_SNR = sum(1 for r in ssap_resultset if r['SNR'] > min_SNR)
print(f"Spectra above threshold: {count_high_SNR}")

## Download High-SNR Spectra
Prompt the user and download the selected files:

In [ ]:
if count_high_SNR == 0:
    print("No spectra exceed the SNR threshold. Exiting.")
else:
    save = input(f"Download {count_high_SNR} spectra with SNR > {min_SNR}? (y/n): ")
    if save.lower() == 'y':
        for row in ssap_resultset:
            if row['SNR'] > min_SNR:
                url = row['access_url'].decode()
                filename = row['CREATORDID'].decode().split('/')[-1]
                print(f"Downloading {filename}...")
                urllib.request.urlretrieve(url, filename)
        print("Download complete.")
    else:
        print("Download canceled by user.")

## Conclusion
In this notebook, we demonstrated how to query the ESO SSA service, apply simple filters, and download data programmatically. This approach can be generalized for other VO services and data filters as needed.